# Предобработка данных

для начала импортируем необходимые библиотеки

In [64]:
import pandas as pd

Откроем наш датасет

In [65]:
df = pd.read_csv("6 Аварии/date.csv")
df.head()

,Unnamed: 0,coordL,coordW,DTPType,date,street,road,NVehicles,NPatricipants,NInjured,...,placeConditions,closeObjects,violations,violationsAdditional,alco,privod,traffic_regime,pokritie,neispravnosti,safety_belt
0,0,37.464817,55.758116,Наезд на препятствие,01.01.2020,ул Мнёвники Нижн.,NaN,1,1,1,...,Перегон (нет объектов на месте ДТП),Остановка общественного транспорта,Несоответствие скорости конкретным условиям дв...,Нет нарушений,NaN,С передним приводом,Режим движения не изменялся,Обработанное противогололедными материалами,Технические неисправности отсутствуют,Нет
1,1,37.388953,55.811338,Наезд на стоящее ТС,01.01.2020,NaN,Московская кольцевая автомобильная дорога (МКА...,2,2,1,...,Перегон (нет объектов на месте ДТП),Отсутствие в непосредственной близости объекто...,Несоответствие скорости конкретным условиям дв...,Нет нарушений,NaN,С передним приводом;С передним приводом,Движение частично перекрыто,Сухое,Технические неисправности отсутствуют;Техничес...,Нет
2,2,37.623153,55.866787,Наезд на пешехода,01.01.2020,NaN,NaN,1,2,1,...,Внутридворовая территория,Административные здания,Несоблюдение очередности проезда,Другие нарушения ПДД водителем,NaN,Иное расположение рулевого управления,Режим движения не изменялся,Сухое,Технические неисправности отсутствуют,Нет
3,3,37.588413,55.660576,Столкновение,01.01.2020,ул Болотниковская,NaN,2,3,1,...,Регулируемый пешеходный переход,Остановка общественного транспорта,Нет нарушений;Несоблюдение очередности проезда,Нет нарушений;Нет нарушений,;,С передним приводом;С передним приводом,Движение частично перекрыто,Мокрое,Технические неисправности отсутствуют;Техничес...,Нет;Нет
4,4,37.636467,55.805098,Падение пассажира,01.01.2020,пр-кт Мира,NaN,1,2,1,...,Остановка общественного транспорта,Остановка общественного транспорта,Неподача или неправильная подача сигналов пово...,Нарушение правил перевозки людей;Нет нарушений,;,С передним приводом,Режим движения не изменялся,Сухое,Технические неисправности отсутствуют,Нет;Нет


### Для начала посмотрим уникальные значения качественных переменных и кол-во каждого типа

Для этого напишем функцию counts

In [66]:
def counts(name: str, df=df):
    series = df[name]
    allValues = {value: (series == value).sum() for value in series.unique()}
    allValues = {k: v for k, v in sorted(allValues.items(), key=lambda item: -item[1])}
    for elem in allValues:
        print(f"{elem}: {allValues[elem]}")

для начала поработаем с lightConditions

In [67]:
df["lightConditions"].head()

0    В темное время суток, освещение включено
1                         Светлое время суток
2                         Светлое время суток
3    В темное время суток, освещение включено
4    В темное время суток, освещение включено
Name: lightConditions, dtype: object

In [68]:
counts("lightConditions")

Светлое время суток: 4624
В темное время суток, освещение включено: 3193
Сумерки: 96
В темное время суток, освещение отсутствует: 36
В темное время суток, освещение не включено: 15


Сумерки можно приравнять к "В темное время суток, освещение включено"

ситуаций с отсутствием освещения было всего 51 из 7964, что составляет чуть больше, чем 0.5% всех данных. Поэтому если приравнять их все к "Светлое время суток", на исследование это не повлияет

In [69]:
changeLight = {
    "Сумерки": "В темное время суток, освещение включено",
    "В темное время суток, освещение не включено": "Светлое время суток",
    "В темное время суток, освещение отсутствует": "Светлое время суток"
}
df["lightConditions"] = df["lightConditions"].replace(changeLight)
counts("lightConditions")

Светлое время суток: 4675
В темное время суток, освещение включено: 3289


Теперь посмотрим на погодные условия

In [70]:
counts("weatherConditions")

Пасмурно: 4294
Ясно: 3261
Дождь: 294
Снегопад: 112
Туман: 3


Заменим Снегопад, Дождь и туман на осадки

In [71]:
df["weatherConditions"] = df["weatherConditions"].replace(["Снегопад", "Дождь", "Туман"], "Осадки")
counts("weatherConditions")

Пасмурно: 4294
Ясно: 3261
Осадки: 409


Теперь посмотрим на состояние дорожного покрытия

In [72]:
counts("pokritie")

Сухое: 4831
Мокрое: 2540
Обработанное противогололедными материалами: 376
Заснеженное: 179
Загрязненное: 19
Гололедица: 13
Со снежным накатом: 4
Свежеуложенная поверхностная обработка: 2


Обработанное противогололедными материалами можно приравнять к сухому

последние 5 типов встречаются суммарно менее чем в 3% случаев. так что можно приравнять к мокрому покрытию

In [73]:
df["pokritie"] = df["pokritie"].replace(["Гололедица", "Заснеженное", "Загрязненное", "Со снежным накатом", "Свежеуложенная поверхностная обработка"], "Мокрое")
df["pokritie"] = df["pokritie"].replace("Обработанное противогололедными материалами", "Сухое")
counts("pokritie")

Сухое: 5207
Мокрое: 2757


рассмотрим переменную, отвечающую за тип привода

In [74]:
counts("privod")

С передним приводом: 2091
С передним приводом;С передним приводом: 1281
С задним приводом: 524
Полноприводные: 406
С передним приводом;Иное расположение рулевого управления: 297
С передним приводом;С задним приводом: 285
Иное расположение рулевого управления: 249
С передним приводом;Полноприводные: 249
Полноприводные;С передним приводом: 242
С задним приводом;С передним приводом: 241
С передним приводом;С передним приводом;С передним приводом: 218
Иное расположение рулевого управления;С передним приводом: 202
С передним приводом;: 173
С задним приводом;С задним приводом: 67
Полноприводные;Иное расположение рулевого управления: 61
Полноприводные;Полноприводные: 54
Иное расположение рулевого управления;Полноприводные: 52
С передним приводом;С передним приводом;Полноприводные: 52
Полноприводные;С задним приводом: 43
;С передним приводом: 42
Полноприводные;С передним приводом;С передним приводом: 41
С передним приводом;С передним приводом;С передним приводом;С передним приводом: 41
С задни

заметим, что тут измерения идут для каждой машины отдельно. так что включать эти данные мы не будем

In [75]:
counts("neispravnosti")

Технические неисправности отсутствуют;Технические неисправности отсутствуют: 3420
Технические неисправности отсутствуют: 3189
Технические неисправности отсутствуют;Технические неисправности отсутствуют;Технические неисправности отсутствуют: 764
Технические неисправности отсутствуют;Технические неисправности отсутствуют;Технические неисправности отсутствуют;Технические неисправности отсутствуют: 192
Иные неисправности: 44
Технические неисправности отсутствуют;Технические неисправности отсутствуют;Технические неисправности отсутствуют;Технические неисправности отсутствуют;Технические неисправности отсутствуют: 44
Коэффициент светопропускания стекол менее нормативного;Технические неисправности отсутствуют: 25
Коэффициент светопропускания стекол менее нормативного: 24
Не устанавливались: 18
Технические неисправности отсутствуют;Коэффициент светопропускания стекол менее нормативного: 15
Наличие конструктивных изменений по сравнению с серийным ТС, сведения о которых отсутствуют в регистрацио

здесь ситуация аналогична

теперь посмотрим информацию о пристёгнутых людях

In [76]:
counts("safety_belt")

Нет: 6499
Нет;Нет: 1189
Нет;Нет;Нет: 140
Да: 24
Нет;Нет;Нет;Нет: 23
Да;Нет: 3
Нет;Да: 3
Нет;Нет;Нет;Нет;Да: 1
Нет;Нет;Нет;Да: 1
Нет;Нет;Нет;Да;Нет;Нет;Нет: 1
Нет;Нет;Нет;Нет;Нет: 1
Нет;Да;Нет: 1
Нет;Нет;Нет;Нет;Нет;Нет: 1
Нет;Нет;Нет;Нет;Нет;Нет;Нет: 1
nan: 0


видно, что значение "да" в выборке есть только у 32 людей и значение нет более чем у 8000. значит эту переменную не имеет смысла рассматривать в нашем исследовании

сохраним только необходимые столбны датасета

In [78]:
columns = ["coordL", "coordW", "placeConditions", "lightConditions", "weatherConditions", "pokritie"]
df = df[columns]
df.head()

,coordL,coordW,placeConditions,lightConditions,weatherConditions,pokritie
0,37.464817,55.758116,Перегон (нет объектов на месте ДТП),"В темное время суток, освещение включено",Пасмурно,Сухое
1,37.388953,55.811338,Перегон (нет объектов на месте ДТП),Светлое время суток,Ясно,Сухое
2,37.623153,55.866787,Внутридворовая территория,Светлое время суток,Пасмурно,Сухое
3,37.588413,55.660576,Регулируемый пешеходный переход,"В темное время суток, освещение включено",Пасмурно,Мокрое
4,37.636467,55.805098,Остановка общественного транспорта,"В темное время суток, освещение включено",Ясно,Сухое


теперь приведём все качественные переменные в логический формат

Сначала lightConditions

In [79]:
counts("lightConditions") 

Светлое время суток: 4675
В темное время суток, освещение включено: 3289


In [80]:
df["lightConditions"] = (df["lightConditions"] == "Светлое время суток").astype('int32')
df = df.rename(columns={"lightConditions": "isLight"})
df.head()

,coordL,coordW,placeConditions,isLight,weatherConditions,pokritie
0,37.464817,55.758116,Перегон (нет объектов на месте ДТП),0,Пасмурно,Сухое
1,37.388953,55.811338,Перегон (нет объектов на месте ДТП),1,Ясно,Сухое
2,37.623153,55.866787,Внутридворовая территория,1,Пасмурно,Сухое
3,37.588413,55.660576,Регулируемый пешеходный переход,0,Пасмурно,Мокрое
4,37.636467,55.805098,Остановка общественного транспорта,0,Ясно,Сухое


теперь weatherConditions

In [81]:
counts("weatherConditions")

Пасмурно: 4294
Ясно: 3261
Осадки: 409


для этой переменной придётся провести one-hot encoding. Оставим переменные Ясно и Пасмурно

In [82]:
isClear = (df["weatherConditions"] == "Ясно").astype('int32')
isCloudy = (df["weatherConditions"] == "Пасмурно").astype('int32')

In [83]:
df["isClear"] = isClear
df["isCloudy"] = isCloudy

In [84]:
df = df.drop('weatherConditions', axis=1)

In [85]:
df.head()

,coordL,coordW,placeConditions,isLight,pokritie,isClear,isCloudy
0,37.464817,55.758116,Перегон (нет объектов на месте ДТП),0,Сухое,0,1
1,37.388953,55.811338,Перегон (нет объектов на месте ДТП),1,Сухое,1,0
2,37.623153,55.866787,Внутридворовая территория,1,Сухое,0,1
3,37.588413,55.660576,Регулируемый пешеходный переход,0,Мокрое,0,1
4,37.636467,55.805098,Остановка общественного транспорта,0,Сухое,1,0


Теперь посмотрим на переменную pokritie

In [87]:
counts("pokritie")

Сухое: 5207
Мокрое: 2757


преобразуем её в isDry

In [88]:
df["pokritie"] = (df["pokritie"] == "Сухое").astype('int32')
df = df.rename(columns={"pokritie": "isDry"})
df.head()

,coordL,coordW,placeConditions,isLight,isDry,isClear,isCloudy
0,37.464817,55.758116,Перегон (нет объектов на месте ДТП),0,1,0,1
1,37.388953,55.811338,Перегон (нет объектов на месте ДТП),1,1,1,0
2,37.623153,55.866787,Внутридворовая территория,1,1,0,1
3,37.588413,55.660576,Регулируемый пешеходный переход,0,0,0,1
4,37.636467,55.805098,Остановка общественного транспорта,0,1,1,0


и, наконец, обработаем placeConditions

In [89]:
counts("placeConditions")

Перегон (нет объектов на месте ДТП): 2810
Нерегулируемый пешеходный переход: 1109
Выезд с прилегающей территории: 749
Регулируемый перекресток: 725
Регулируемый пешеходный переход: 623
Внутридворовая территория: 497
Нерегулируемый перекрёсток неравнозначных улиц (дорог): 429
Мост, эстакада, путепровод: 297
Остановка общественного транспорта: 245
Нерегулируемый перекрёсток равнозначных улиц (дорог): 81
Тротуар, пешеходная дорожка: 73
Автостоянка (отделенная от проезжей части): 62
Подход к мосту, эстакаде, путепроводу: 61
Тоннель: 56
Автостоянка (не отделённая от проезжей части): 37
Остановка трамвая: 30
АЗС: 22
Иное место: 18
Нерегулируемое пересечение с круговым движением: 14
Нерегулируемый пешеходный переход, расположенный на участке улицы или дороги, проходящей вдоль территории школы или иной детской организации: 11
Пешеходная зона: 5
Гаражные постройки (гаражный кооператив, товарищество либо иное место концентрированного размещения гаражей): 5
Регулируемый ж/д переезд с дежурным: 2


разобьём это на несколько признаков. группировка будет производиться по признако является ли место зоной скопления пешеходов и регулировалось ли знаками место

In [97]:
pedestrian_places = ["Регулируемый пешеходный переход",
                    "Нерегулируемый пешеходный переход",
                    "Внутридворовая территория",
                    "Остановка общественного транспорта",
                    "Тротуар, пешеходная дорожка",
                    "Подход к мосту, эстакаде, путепроводу",
                    "Остановка трамвая",
                    "Нерегулируемый пешеходный переход, расположенный на участке улицы или дороги, проходящей вдоль территории школы или иной детской организации",
                    "Пешеходная зона",
                    "Гаражные постройки (гаражный кооператив, товарищество либо иное место концентрированного размещения гаражей)",
                    "Регулируемый пешеходный переход, расположенный на участке улицы или дороги, проходящей вдоль территории школы или иной детской организации",
                     "Остановка маршрутного такси"]
is_pedestrian =(df["placeConditions"].isin(pedestrian_places)).astype('int32')

In [98]:
adjustable_places = ["Регулируемый перекресток",
                    "Регулируемый пешеходный переход",
                    "Мост, эстакада, путепровод",
                    "Подход к мосту, эстакаде, путепроводу",
                    "Регулируемый ж/д переезд с дежурным",
                    "Регулируемый пешеходный переход, расположенный на участке улицы или дороги, проходящей вдоль территории школы или иной детской организации",
                    "Регулируемый ж/д переезд без дежурного"]
is_adjustable = (df["placeConditions"].isin(adjustable_places)).astype('int32')

In [100]:
df["is_pedestrian"] = is_pedestrian
df["is_adjustable"] = is_adjustable

df = df.drop('placeConditions', axis=1)
df.head()

,coordL,coordW,isLight,isDry,isClear,isCloudy,is_pedestrian,is_adjustable
0,37.464817,55.758116,0,1,0,1,0,0
1,37.388953,55.811338,1,1,1,0,0,0
2,37.623153,55.866787,1,1,0,1,1,0
3,37.588413,55.660576,0,0,0,1,1,1
4,37.636467,55.805098,0,1,1,0,1,0


Все переменные, кроме координат привидены к нужному виду. На том предобработка данных закончена

In [101]:
df.to_csv("6 Аварии/transformation.csv")